# Exercises week 6

Exercises about types.

**Note:** You must specify the type for all the created functions.

**Exercise 1.** Make the Date type from previous week exercises to derive from `Ord` and redefine the ordering operators (< > >= <=) so they work properly.

In [1]:
data Date = Date {day::Int, month::Int, year::Int} deriving (Show, Eq)

instance Ord Date where
    dt1@(Date d1 m1 y1) >  dt2@(Date d2 m2 y2) = isPrevious dt2 dt1
    dt1@(Date d1 m1 y1) <  dt2@(Date d2 m2 y2) = isPrevious dt1 dt2
    dt1@(Date d1 m1 y1) >= dt2@(Date d2 m2 y2) = isPrevious dt2 dt1 || dt2 == dt1
    dt1@(Date d1 m1 y1) <=  dt2@(Date d2 m2 y2) = isPrevious dt1 dt2 || dt2 == dt1

-- We will reuse the isPrevious function
isPrevious::Date -> Date -> Bool
isPrevious (Date d1 m1 y1) (Date d2 m2 y2) 
    | y1 Prelude.> y2 = False
    | y1 Prelude.< y2 = True
    | m1 Prelude.> m2 = False
    | m1 Prelude.< m2 = True
    | d1 Prelude.< d2 = True
    | otherwise = False

In [2]:
Date 31 1 2022 > Date 30 1 2022
Date 30 1 2022 > Date 30 1 2022
Date 31 1 2022 >= Date 30 1 2022
Date 30 1 2022 <= Date 30 1 2022

True

False

True

True

**Exercise 2.** Make the Date type to derive from Enum as well, implementing `succ` and `pred` functions.

In [3]:
instance Enum Date where
succ dt@(Date d m y) = nextDay dt
pred (Date d m y ) 
    | rightDay (Date (d - 1) m y) = Date (d - 1) m y
    | rightMonth (Date d (m - 1) y) = Date (maxDays (Date d (m - 1) y)) (m - 1) y
    | otherwise = Date 31 12 (y - 1)

-- To calculate the maximum number of days of a month
maxDays::Date -> Int
maxDays (Date d m y)
    | m `elem` [1, 3, 5, 7, 8, 10, 12] = 31
    | m `elem` [4, 6, 9, 11] = 30
    | m == 2 = if isleap (Date d m y) then 29 else 28
    | otherwise = error "Wrong month"

--- Reusing some functions
nextDay::Date -> Date
nextDay (Date d m y) 
    | rightDay (Date (d + 1) m y) = Date (d + 1) m y
    | rightMonth (Date 1 (m + 1) y) = Date 1 (m + 1) y
    | otherwise = Date 1 1 (y + 1)
    
isleap::Date -> Bool
isleap (Date _ _ y) = y `mod` 4 == 0 && (y `mod` 100 /= 0 || y `mod` 400 == 0)

rightMonth::Date -> Bool
rightMonth (Date _ m _) = m Prelude.> 0 && m Prelude.< 13

rightDay::Date -> Bool
rightDay (Date d m y)
    | d Prelude.< 1 || d Prelude.> 31 = False
    | m `elem` [1, 3, 5, 7, 8, 10, 12] = True
    | m == 2 = d Prelude.< 29 || (d == 29 && isleap (Date d m y))
    | m `elem` [4, 6, 9, 11] = d Prelude.<= 30
    | otherwise = error "Wrong month"

In [4]:
succ (Date 31 12 2022)
succ (Date 31 1 2022)
pred (Date 31 12 2022)
pred (Date 1 1 2022)
pred (Date 1 12 2022)
pred (Date 1 3 2022)
pred (Date 1 3 2020)

Date {day = 1, month = 1, year = 2023}

Date {day = 1, month = 2, year = 2022}

Date {day = 30, month = 12, year = 2022}

Date {day = 31, month = 12, year = 2021}

Date {day = 30, month = 11, year = 2022}

Date {day = 28, month = 2, year = 2022}

Date {day = 29, month = 2, year = 2020}

**Exercise 3.** Create a Product type with two constructors. The first one, named Product, will receive the name, the price and the units in stock of that product. The second one, FreshProduct, will receive also a Date (see last week exercises) with the expiration date.

In [5]:
data Product = Product {name::String, price::Float, stock::Int} 
                | FreshProduct {name::String, price::Float, stock::Int, expiration::Date} deriving (Show, Eq)

In [6]:
Product "shoes" 12.99 45
FreshProduct "oranges" 1.18 33 (Date 1 1 2023)

Product {name = "shoes", price = 12.99, stock = 45}

FreshProduct {name = "oranges", price = 1.18, stock = 33, expiration = Date {day = 1, month = 1, year = 2023}}

**Exercise 4.** Create a `sell Product` function that sells a unit of a product if its stock is bigger than 0, it returns the product with the new stock

In [7]:
sell::Product -> Product
sell (Product n p s) 
    | s > 0 = Product n p (s - 1)
    | otherwise = Product n p s
sell (FreshProduct n p s (Date d m y))
    | s > 0 = FreshProduct n p (s - 1) (Date d m y)
    | otherwise = FreshProduct n p s (Date d m y)

In [8]:
sell (Product "shoes" 12.99 45)
sell (FreshProduct "oranges" 1.18 33 (Date 1 1 2023))

Product {name = "shoes", price = 12.99, stock = 44}

FreshProduct {name = "oranges", price = 1.18, stock = 32, expiration = Date {day = 1, month = 1, year = 2023}}

**Exercise 5.** Create a function `expired Date Product` that returns if the product has expired

In [9]:
isPrevious::Date -> Date -> Bool
isPrevious (Date d1 m1 y1) (Date d2 m2 y2) 
    | y1 > y2 = False
    | y1 < y2 = True
    | m1 > m2 = False
    | m1 < m2 = True
    | d1 < d2 = True
    | otherwise = False

expired::Date -> Product -> Bool
expired (Date d m y) (FreshProduct _ _ _ (Date d2 m2 y2)) = not (isPrevious (Date d m y) (Date d2 m2 y2))
expired _ Product {} = False

In [10]:
expired (Date 31 1 2023) (Product "shoes" 12.99 45)
expired (Date 31 1 2023) (FreshProduct "oranges" 1.18 33 (Date 1 1 2023))

False

True

**Exercise 6.** Redefine the == operator as follows:
1. Two Products are equal if their names and prices are equal
2. Two FreshProducts are equal if in addition their expiration dates are equal

In [11]:
instance Eq Product where
    Product n1 p1 _ == Product n2 p2 _ = n1 Prelude.== n2 && p1 Prelude.== p2
    FreshProduct n1 p1 _ (Date d1 m1 y1) == FreshProduct n2 p2 _ (Date d2 m2 y2) = 
        n1 Prelude.== n2 && p1 Prelude.== p2 && Date d1 m1 y1 Prelude.== Date d2 m2 y2

In [12]:
Product "shoes" 12.99 45 == Product "shoes" 12.99 55
Product "shoes" 12.99 45 == Product "shoes" 1.99 55
FreshProduct "oranges" 1.18 33 (Date 1 1 2023) == FreshProduct "oranges" 1.18 12 (Date 1 1 2023)
FreshProduct "oranges" 1.18 33 (Date 1 1 2023) == FreshProduct "oranges" 1.18 12 (Date 12 1 2023)
FreshProduct "oranges" 1.18 33 (Date 1 1 2023) /= FreshProduct "oranges" 1.18 12 (Date 12 1 2023)

True

False

True

False

True

# Delivery rules

Save this notebook with name `your initials-week-06.ipynb` and upload it to Aula Global before the deadline. For example, Lucía Perez Gómez will upload a file `lpg-week-06.ipynb`